In [ ]:
%pip install srai[overturemaps] openpyxl contextily seaborn

# Predict population density

In this notebook we will aim to predict the population density in the London MSOA (Middle layer Super Output Area) regions.

We will use open data form the London datastore and try to predict the population using features from the Overture Maps dataset.

<a target="_blank" href="https://colab.research.google.com/github/kraina-ai/srai-tutorial/blob/geopython2025/tutorial/01_london_population_density_prediction.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Prepare London population dataset

We will download geometries for the MSOA (Middle layer Super Output Area) regions for London from 2021 and combine it with population estimates for 2022.

Based on that we will calculate population density and try to predict it.

In [ ]:
import zipfile
from pathlib import Path

import contextily as cx
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd
import seaborn as sns
from pooch import retrieve
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from srai.embedders import CountEmbedder
from srai.loaders import OvertureMapsLoader

### Download geometries for the MSOA regions

In [ ]:
msoa_url = "https://data.london.gov.uk/download/statistical-gis-boundary-files-london/f6d9340a-2ccb-46ad-846b-c9122b4b5d1f/LB_MSOA2021_shp.zip"
destination_file = retrieve(
    url=msoa_url,
    fname=Path(msoa_url).name,
    path=".",
    known_hash=None,
)

In [ ]:
zip_path = destination_file
zip_paths = []

with zipfile.ZipFile(zip_path, mode="r") as archive:
    zip_paths = [
        f"zip:{zip_path}!{p}" for p in archive.namelist() if p.endswith(".shp")
    ]

zip_paths

### Load MSOA boundaries

We will load each SHP from inside the zip file and concatenate the results.

In [ ]:
msoa_gdf = gpd.pd.concat(
    [gpd.read_file(p, columns=["msoa21cd", "msoa21nm", "geometry"]) for p in zip_paths]
)
msoa_gdf

Notice the coordinates in the geometries aren't in the WGS84 CRS.

They are saved in the projected CRS EPSG:27700, which is the British National Grid, with units in meters.

In [ ]:
msoa_gdf.crs

### Download and load MSOA statistics

We will load the statistics for the year of 2021.

In [ ]:
stats = "https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/middlesuperoutputareamidyearpopulationestimatesnationalstatistics/mid2021andmid2022/sapemsoaquinaryagetablefinal.xlsx"
destination_file = retrieve(
    url=stats,
    fname="msoa_density.xlsx",
    path=".",
    known_hash=None,
)

x = pd.read_excel(
    destination_file,
    sheet_name="Mid-2022 MSOA 2021",
    skiprows=3,
)[["MSOA 2021 Code", "Total"]]
x

### Combine geometries with statistics

Here we will combine geometries GeoDataFrame with population per region.

Then we will calculate the area (since the geometries are in the projected CRS, it will calculate are properly in meters squared).

After calculating the area we can change the CRS to WGS84 (or EPSG:4326).

In [ ]:
msoa_stats_gdf = msoa_gdf.merge(x, left_on="msoa21cd", right_on="MSOA 2021 Code")
msoa_stats_gdf["area"] = msoa_stats_gdf.area
msoa_stats_gdf["population_density"] = msoa_stats_gdf["Total"] / msoa_stats_gdf["area"]
msoa_stats_gdf = msoa_stats_gdf.to_crs(epsg=4326).set_index("msoa21cd")
msoa_stats_gdf

### Plot the density

Let's see how the density is distributed across the city.

In [ ]:
f, ax = plt.subplots(figsize=(12, 8))
msoa_stats_gdf.plot("population_density", ax=ax, legend=True, alpha=0.8, cmap="plasma")
cx.add_basemap(ax, crs=msoa_stats_gdf.crs, source=cx.providers.CartoDB.PositronNoLabels)
ax.set_axis_off()

plt.show()

## Generate embeddings

Let's now download the Overture Maps data and prepare some features for each region.

### Download the data using OvertureMapsLoader

In [ ]:
london_features = OvertureMapsLoader(hierarchy_depth=1).load(msoa_stats_gdf)
london_features

### Calculate intersections

To properly calculate which type of feature is in region, we need to know which geometries intersect between two datasets.

Here we will manually create and query spatial the STRTree index. In other examples we will use `IntersectionJoiner` from the srai library.

Because `CountEmbedder` expects dataframe as input, we will create an empty DataFrame with just an index.

In [ ]:
region_idx, features_idx = london_features.sindex.query(
    msoa_stats_gdf.geometry, predicate="intersects"
)
features_per_msoa = pd.DataFrame(
    index=pd.MultiIndex.from_arrays(
        (msoa_stats_gdf.index[region_idx], london_features.index[features_idx]),
        names=(msoa_stats_gdf.index.name, london_features.index.name),
    )
)
features_per_msoa

### Explore the input data

In [ ]:
city_of_london_msoa_code = "E02000001"

msoa_area = msoa_stats_gdf.loc[[city_of_london_msoa_code]]
msoa_area_bounds = msoa_area.total_bounds

msoa_features = london_features.loc[
    features_per_msoa.loc[city_of_london_msoa_code].index
]

f, ax = plt.subplots(figsize=(12, 8))

msoa_stats_gdf.loc[[city_of_london_msoa_code]].boundary.plot(
    ax=ax, color="black", alpha=0.8, lw=1
)

# Plot polygons
# We will remove Polygon features that aren't fully within the area for clarity.
msoa_features[
    msoa_features.geom_type.isin(("Polygon", "MultiPolygon"))
    & msoa_features.within(msoa_area.union_all())
].plot(
    ax=ax,
    color=(0, 0, 0, 0),
    lw=0.5,
    hatch="//",
    edgecolor="orange",
)

# Plot linestrings
msoa_features[msoa_features.geom_type == "LineString"].plot(
    ax=ax, color="royalblue", alpha=0.8, lw=0.3
)

# Plot points
msoa_features[msoa_features.geom_type == "Point"].plot(
    ax=ax, color="navy", alpha=0.2, markersize=1
)

ax.set_xlim(msoa_area_bounds[0] - 1e-3, msoa_area_bounds[2] + 1e-3)
ax.set_ylim(msoa_area_bounds[1] - 1e-3, msoa_area_bounds[3] + 1e-3)

cx.add_basemap(
    ax, crs=msoa_stats_gdf.crs, source=cx.providers.CartoDB.PositronNoLabels, zoom=15
)
ax.set_axis_off()

plt.show()

### Calculate the embedding

Here we will transform the data and return a count of features in each region.

We will use the `CountEmbedder` with optimized implementation, but you can try to do it manually.

In [ ]:
embeddings = CountEmbedder(count_subcategories=False).transform(
    msoa_stats_gdf, london_features, features_per_msoa
)
embeddings

## Train the model

Now we can use calculated embeddings and our target column to train a simple model.

In [ ]:
train_msoa_codes, test_msoa_codes = train_test_split(
    msoa_stats_gdf.index,
    test_size=0.33,
    random_state=42,
)

X_train = embeddings.loc[train_msoa_codes]
X_test = embeddings.loc[test_msoa_codes]
y_train = msoa_stats_gdf.loc[train_msoa_codes, "population_density"]
y_test = msoa_stats_gdf.loc[test_msoa_codes, "population_density"]

In [ ]:
model = RandomForestRegressor()
model.fit(X_train, y_train)
model

In [ ]:
y_pred = model.predict(X_test)

r2_score(y_test, y_pred)

### Compare the predictions with original data

In [ ]:
f, ax = plt.subplots(figsize=(8, 8))

sns.regplot(
    x=y_test,
    y=y_pred,
    scatter_kws=dict(alpha=0.5, s=10),
    line_kws=dict(color=".2", linestyle="--"),
    ax=ax,
)
min_density = y_test.min()
max_density = y_test.max()
sns.lineplot(
    x=[min_density, max_density], y=[min_density, max_density], color="red", ax=ax
)

ax.set_xlabel("True population density")
ax.set_ylabel("Predicted population density")

plt.show()

### Plot feature importance for the model

In [ ]:
ax = sns.barplot(
    pd.DataFrame(
        {
            "feature_importance": model.feature_importances_,
            "feature_names": embeddings.columns,
        }
    ).nlargest(20, "feature_importance"),
    y="feature_names",
    x="feature_importance",
)
ax.set_title("Feature importances")
ax.set_ylabel("Features")
ax.set_xlabel("Feature importance")
plt.show()

### Predict the data for the whole dataset

In [ ]:
msoa_stats_gdf["predicted_population_density"] = model.predict(
    embeddings.loc[msoa_stats_gdf.index]
)

### Explore the results

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 16), sharex=True, sharey=True)
msoa_stats_gdf.plot("population_density", ax=ax1, legend=True, cmap="plasma")
msoa_stats_gdf.plot("predicted_population_density", ax=ax2, legend=True, cmap="plasma")

ax1.set_title("Population density")
ax2.set_title("Predicted population density")

plt.show()

Calculate the prediction error and visualize it on a map.

In [ ]:
msoa_stats_gdf["error"] = (
    msoa_stats_gdf["population_density"]
    - msoa_stats_gdf["predicted_population_density"]
)

# Rescale from min / max to 0 - 1 with center at 0.5
msoa_stats_gdf["normalized_error"] = (
    msoa_stats_gdf["error"].apply(
        lambda x, city_data=msoa_stats_gdf: (
            -x / city_data["error"].min()
            if x < 0
            else x / city_data["error"].max()
        )
    )
    + 1
) / 2

msoa_stats_gdf["alpha"] = (
    msoa_stats_gdf["normalized_error"] - 0.5
).abs() * 2

In [ ]:
f, ax = plt.subplots(figsize=(12, 8))

msoa_stats_gdf.boundary.plot(ax=ax, color="black", alpha=0.8, lw=0.1)
msoa_stats_gdf.plot(
    "normalized_error",
    ax=ax,
    alpha=msoa_stats_gdf["alpha"],
    cmap="bwr_r",
    legend=True,
    legend_kwds={
        "shrink": 0.8,
        "label": "Prediction error",
        "ticks": [0, 0.5, 1],
        "format": mticker.FixedFormatter(
            [
                round(msoa_stats_gdf["error"].min(), 4),
                "0",
                round(msoa_stats_gdf["error"].max(), 4),
            ]
        ),
    },
)

cx.add_basemap(
    ax, crs=msoa_stats_gdf.crs, source=cx.providers.CartoDB.PositronNoLabels, zoom=12
)
ax.set_axis_off()

plt.show()

## Task for you

- Transform the predicted population density back to a total number of residents.
- Compare the results using a total number of residents and not population density

In [ ]:
# Write code here

# You can find answers here: https://github.com/kraina-ai/srai-tutorial/tree/geopython2025/answers